In [183]:
#Import Packages 
import pyodbc
import pandas as pd
import numpy as np
import sys, os, re, os.path
import base64
import requests
from datetime import datetime
from datetime import date
import shutil
from shutil import SameFileError
import ftplib
from ftplib import FTP #to connect to an FTP server
from crontab import CronTab
import csv
import subprocess
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import win32com.client as client
from bs4 import BeautifulSoup
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Configuration
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [184]:
#%pip install pywin32
#%pip install crontab
#%pip install --upgrade pywin32==225 
#print(pywin32.__version__)

# import platform
# print(platform.architecture())

# import win32api

In [185]:
conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=USODCVSQL0256,7114;'
                        'Database=MI_LEADS_OUTPUT;'
                        'UID=MISA;'
                        'PWD=Sra0!tmMz;'
                        'Trusted_Connection=no;')


df = pd.read_sql_query('SELECT * FROM [LEADS_LIST_FILE_CREATE_REQUEST]', conn)
df.head(10)

,ROW_ID,TABLE_NAME,PROCESSED,DATETIME_PROCESSED,TACTIC,CAMPAIGN_NAME,VENDOR,CAMPAIGN_DATE,DISTRIBUTION_LIST
0,1,AB97883_DM_FINAL_TEST,0,None,DM,Test Campaign,INT,2023-08-08,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
1,2,1DMRRD_EN23X_10122022_B44,0,None,DM,eBill Notification,RRD,2023-08-08,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
2,3,1DRCYD_QW_WAD9_10062022_6C6,0,None,D2D,None,None,None,None
3,4,1EMSMC_EN00Y_10132022,0,None,EM,eBill Notification,SMC,2023-08-08,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
4,5,1EMSMC_UJ00Y_10132022,0,None,EM,New_or_Upg_UO,SMC,2023-08-08,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
5,6,1TMVXI_NP24Y_10102022_145,0,None,TM,NPS_Detractor,VXI,2023-08-08,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
6,7,1DHINT_RG23XUT_LOCAL10112022_420,0,None,DH,UT_LOCAL_Dec,INT,2023-08-08,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
7,8,1DMRRD_EN23X_08092023_B22,0,None,DM,eBill Notification,RRD,2023-08-09,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
8,9,1DMRRD_EN23X_08162023_4F8,0,None,DM,eBill Notification,RRD,2023-08-16,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM


In [190]:
def leadscreation_and_email(dbcon):
    
    # instantiate outlook object 
    outlook = client.Dispatch("Outlook.Application") # to dispatch any of the office products you installed on the computer 
    
    # connect to the server USODCVSQL0256,7114 at db MI_LEADS and see if there're tables in the file creation driver table
    dbcur = dbcon.cursor()
    print("Connected to SQL Server Successfully!",'\n\n')
        
    dbcur.execute("""
                  select count(*)
                  from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                  """) 
    
    if dbcur.fetchone()[0] > 0:
        print('There are files/tables in the driver table!','\n\n') 
        
        # Read table and check if column contains processed = 1 and DATETIME_PROCESSED, if contains 1 then ignore and only process processed = 0 
        # populate file creation driver table
        sql = str("""select * 
                     from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                      --where PROCESSED = 0 and DATETIME_PROCESSED is null
                      ; -- static table
                  """)
                
        df = pd.DataFrame(pd.read_sql(sql,con = dbcon))
        print(df['TACTIC'].tolist(),'\n\n')
        
        
        for ind, row in df.iterrows():  #print(row['PROCESSED'], row['TACTIC'], row['TABLE_NAME'])
            if (row['PROCESSED'] == 0):
                if (row['TACTIC'].strip() == 'DM'):
                    
                    # iterate rows in dataframe to identify different tactics and export files in different format
                    dm_sql = str("""select distinct TABLE_NAME
                                    from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where PROCESSED = 0 
                                    and DATETIME_PROCESSED is null
                                    and TACTIC = 'DM'
                                    and TABLE_NAME = '{0}'
                                    ; -- static table
                                 """).format(row['TABLE_NAME'])
                    dm_table_name = pd.DataFrame(pd.read_sql(dm_sql, con = dbcon))
                    print(dm_table_name['TABLE_NAME'].tolist(),'\n\n')
                    
                    # read in table and output files to source folders 
                    for dm_tablename in dm_table_name['TABLE_NAME'].tolist():
                        read_dm_table_sql = str("""select * from [{0}];
                                                """).format(dm_tablename)
                        df_each_dm_table = pd.read_sql(read_dm_table_sql, con = dbcon) 
                        df_each_dm_table.to_csv('//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing/' + dm_tablename + date.today().strftime("%m%d%Y")+'.txt',  sep='|', header=True, index=False)
                    
                        # update table records with processed = 1 and system datetime stamp 
                        update_dm_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (dm_tablename))
                        dbcon.commit()
                        
                        # generate counts for email report
                        email_dm_table_sql = str("""select SPR_NBR as KEYCODE, count(*) as COUNT
                                                    from MI_LEADS_OUTPUT.[dbo].[{0}]
                                                    group by SPR_NBR;
                                                """).format(dm_tablename)
                        df_email_each_dm_table = pd.read_sql(email_dm_table_sql, con = dbcon) 
                        dm_html_table = df_email_each_dm_table.to_html(index=False) # transform table in html format
                        
                        # send email notifications
                        mail = outlook.CreateItem(0)
                        mail.To = row["DISTRIBUTION_LIST"]
                        mail.Subject = f"{row['CAMPAIGN_NAME']} {row['TACTIC']} LIST {row['CAMPAIGN_DATE']} ct:{df_email_each_dm_table.COUNT.sum()}"
                        mail.HTMLBody = f"""
                                            <b>Team, <br><br>
                                            I am sending the {row['CAMPAIGN_NAME']} leads to you via ftp. The details of the file are listed below. <br><br>
                                            Counts for this campaign are provided below.<br><br>
                                            File name:<br><br>
                                            ---------------------------------------------------<br><br>
                                            {dm_tablename}.TXT  - {df_email_each_dm_table.COUNT.sum()} records <br><br>
                                            {dm_html_table}
                                            
                                         """

                        #mail.Display()
                        mail.Send()

           
                if (row['TACTIC'].strip() == 'TM'):
                    tm_sql = str("""select distinct TABLE_NAME
                                    from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where PROCESSED = 0 
                                    and DATETIME_PROCESSED is null
                                    and TACTIC = 'TM'
                                    and TABLE_NAME = '{0}'
                                    ; -- static table
                                 """).format(row['TABLE_NAME'])       
                    tm_table_name = pd.DataFrame(pd.read_sql(tm_sql, con = dbcon))
                    print(tm_table_name['TABLE_NAME'].tolist(),'\n\n')
                
                    for tm_tablename in tm_table_name['TABLE_NAME'].tolist():
                        read_tm_table_sql = str("""select * from [{0}];
                                                """).format(tm_tablename)
                        df_each_tm_table = pd.read_sql(read_tm_table_sql, con = dbcon) 
                        df_each_tm_table.to_csv('//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing/' + tm_tablename + date.today().strftime("%m%d%Y")+'.txt',  sep='?', header=True, index=False)
                    
                        # update table records with processed = 1 and system datetime stamp 
                        update_tm_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (tm_tablename))
                        dbcon.commit()  

                    
                        # generate counts for email report
                        email_tm_table_sql = str("""select CAMPAIGN_CODE as KEYCODE, count(*) as COUNT
                                                   from MI_LEADS_OUTPUT.[dbo].[{0}]
                                                   group by CAMPAIGN_CODE;
                                                """).format(tm_tablename)
                        df_email_each_tm_table = pd.read_sql(email_tm_table_sql, con = dbcon) 
                        tm_html_table = df_email_each_tm_table.to_html(index=False) # transform table in html format
                        
                        # send email notifications
                        mail = outlook.CreateItem(0)
                        mail.To = row["DISTRIBUTION_LIST"]
                        mail.Subject = f"{row['CAMPAIGN_NAME']} {row['TACTIC']} LIST {row['CAMPAIGN_DATE']} ct:{df_email_each_tm_table.COUNT.sum()}"
                        mail.HTMLBody = f"""
                                            <b>Team, <br><br>
                                            I am sending the {row['CAMPAIGN_NAME']} leads to you via ftp. The details of the file are listed below. <br><br>
                                            Counts for this campaign are provided below.<br><br>
                                            File name:<br><br>
                                            ---------------------------------------------------<br><br>
                                            {tm_tablename}.TXT  - {df_email_each_tm_table.COUNT.sum()} records <br><br>
                                            {tm_html_table}
                                            
                                         """

                        #mail.Display()
                        mail.Send()



                if (row['TACTIC'].strip() == 'EM'):
                    em_sql = str("""select distinct TABLE_NAME
                                    from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where PROCESSED = 0 
                                    and DATETIME_PROCESSED is null
                                    and TACTIC = 'EM'
                                    and TABLE_NAME = '{0}'                                    
                                    ; -- static table
                                  """).format(row['TABLE_NAME'])
                    em_table_name = pd.DataFrame(pd.read_sql(em_sql, con = dbcon))
                    print(em_table_name['TABLE_NAME'].tolist(),'\n\n')
                
                    for em_tablename in em_table_name['TABLE_NAME'].tolist():
                        read_em_table_sql = str("""select * from [{0}];
                                                """).format(em_tablename)
                        df_each_em_table = pd.read_sql(read_em_table_sql, con = dbcon) 
                        df_each_em_table.to_csv('//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing/' + em_tablename + date.today().strftime("%m%d%Y")+'.csv',  sep=',', header=True, index=False)
               
                        # update table records with processed = 1 and system datetime stamp 
                        update_em_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (em_tablename))
                        dbcon.commit()           


                    
                        # generate counts for email report
                        email_em_table_sql = str("""select CAMPAIGN_CODE as KEYCODE, count(*) as COUNT
                                                   from MI_LEADS_OUTPUT.[dbo].[{0}]
                                                   group by CAMPAIGN_CODE;
                                                """).format(em_tablename)
                        df_email_each_em_table = pd.read_sql(email_em_table_sql, con = dbcon) 
                        em_html_table = df_email_each_em_table.to_html(index=False) # transform table in html format
                        
                        # send email notifications
                        mail = outlook.CreateItem(0)
                        mail.To = row["DISTRIBUTION_LIST"]
                        mail.Subject = f"{row['CAMPAIGN_NAME']} {row['TACTIC']} LIST {row['CAMPAIGN_DATE']} ct:{df_email_each_em_table.COUNT.sum()}"
                        mail.HTMLBody = f"""
                                            <b>Team, <br><br>
                                            I am sending the {row['CAMPAIGN_NAME']} leads to you via ftp. The details of the file are listed below. <br><br>
                                            Counts for this campaign are provided below.<br><br>
                                            File name:<br><br>
                                            ---------------------------------------------------<br><br>
                                            {em_tablename}.TXT  - {df_email_each_em_table.COUNT.sum()} records <br><br>
                                            {em_html_table}
                                            
                                         """

                        #mail.Display()
                        mail.Send()


                        
                if (row['TACTIC'].strip() == 'D2D'):
                    D2D_sql = str("""select distinct TABLE_NAME
                                      from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                      where PROCESSED = 0 
                                      and DATETIME_PROCESSED is null
                                      and TACTIC = 'D2D'
                                      and TABLE_NAME = '{0}'   
                                      ; -- static table
                                  """).format(row['TABLE_NAME'])
                    D2D_table_name = pd.DataFrame(pd.read_sql(D2D_sql, con = dbcon))
                    print(D2D_table_name['TABLE_NAME'].tolist(),'\n\n')
                
                    for D2D_tablename in D2D_table_name['TABLE_NAME'].tolist():
                        read_d2d_table_sql = str("""select * from [{0}];
                                                 """).format(D2D_tablename)
                        df_each_d2d_table = pd.read_sql(read_d2d_table_sql, con = dbcon) 
                        df_each_d2d_table.to_csv('//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing/' + D2D_tablename + date.today().strftime("%m%d%Y")+'.csv',  sep=',', header=True, index=False)
                
                        # update table records with processed = 1 and system datetime stamp 
                        update_d2d_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (D2D_tablename))
                        dbcon.commit()   
       
        
                if (row['TACTIC'].strip() == 'DH'):
                    DH_sql = str("""select distinct TABLE_NAME
                                     from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                     where PROCESSED = 0 
                                     and DATETIME_PROCESSED is null
                                     and TACTIC = 'DH'
                                     and TABLE_NAME = '{0}'  
                                     ; -- static table
                                   """).format(row['TABLE_NAME'])
                    DH_table_name = pd.DataFrame(pd.read_sql(DH_sql, con = dbcon))
                    print(DH_table_name['TABLE_NAME'].tolist(),'\n\n')
                
                    for DH_tablename in DH_table_name['TABLE_NAME'].tolist():
                        read_dh_table_sql = str("""select * from [{0}];
                                                """).format(DH_tablename)
                            
                        df_each_dh_table = pd.read_sql(read_dh_table_sql, con = dbcon) 
                        df_each_dh_table.to_csv('//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing/' + DH_tablename + date.today().strftime("%m%d%Y")+'.csv',  sep=',', header=True, index=False)

                        # update table records with processed = 1 and system datetime stamp 
                        update_dh_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (DH_tablename))
                        dbcon.commit()   


                    
                        # generate counts for email report
                        email_dh_table_sql = str("""select SPR_NBR as KEYCODE, count(*) as COUNT
                                                   from MI_LEADS_OUTPUT.[dbo].[{0}]
                                                   group by SPR_NBR;
                                                """).format(DH_tablename)
                        df_email_each_dh_table = pd.read_sql(email_dh_table_sql, con = dbcon) 
                        dh_html_table = df_email_each_dh_table.to_html(index=False) # transform table in html format
                        
                        # send email notifications
                        mail = outlook.CreateItem(0)
                        mail.To = row["DISTRIBUTION_LIST"]
                        mail.Subject = f"{row['CAMPAIGN_NAME']} {row['TACTIC']} LIST {row['CAMPAIGN_DATE']} ct:{df_email_each_dh_table.COUNT.sum()}"
                        mail.HTMLBody = f"""
                                            <b>Team, <br><br>
                                            I am sending the {row['CAMPAIGN_NAME']} leads to you via ftp. The details of the file are listed below. <br><br>
                                            Counts for this campaign are provided below.<br><br>
                                            File name:<br><br>
                                            ---------------------------------------------------<br><br>
                                            {DH_tablename}.TXT  - {df_email_each_dh_table.COUNT.sum()} records <br><br>
                                            {dh_html_table}
                                            
                                         """

                        #mail.Display()
                        mail.Send()

   
            elif row['PROCESSED'] == 1:
                print('Table have already been processed, please check back later!')
                
            else:
                print('Table have already been processed, please check back later!')


    
    else:
        print('There are no files/tables in the driver table!')
       # return False
    
    
    # move files to destination folder #\CCM_AUTOMATION\Adhoc\Output
    src_files = os.listdir(r"//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing")
    src = r"//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing"
    dest = r"Y:\CCM_AUTOMATION\Adhoc\Output\Testing_Files_JH" #r"C:\\Users\\AC27802\OneDrive - CenturyLink\Desktop\test_sh\copy_test_files" -- need to use mapped network drive 
     
    try:
        for file_name in src_files:
            dest_full_file_name = os.path.join(dest, file_name)
            src_full_file_name = os.path.join(src, file_name)
            if os.path.isfile(src_full_file_name):
                shutil.copy(src_full_file_name, dest)
                print("Source", src_full_file_name,'\n\n'
                      "Destination",dest_full_file_name,'\n\n'
                      "copied to destination successfully",'\n\n') 

    except IOError:
        print("Error raised with moving files to destination!",'\n\n') 
    else:
        print("Source file copied to destination successfully!",'\n\n')

   # dbcon.close() # close server connection 

In [191]:
leadscreation_and_email(conn)

Connected to SQL Server Successfully! 


There are files/tables in the driver table! 


['DM ', 'DM ', 'D2D', 'EM ', 'EM ', 'TM ', 'DH ', 'DM ', 'DM '] 


['AB97883_DM_FINAL_TEST'] 


['1DMRRD_EN23X_10122022_B44'] 


['1DRCYD_QW_WAD9_10062022_6C6'] 


['1EMSMC_EN00Y_10132022'] 


['1EMSMC_UJ00Y_10132022'] 


['1TMVXI_NP24Y_10102022_145'] 


['1DHINT_RG23XUT_LOCAL10112022_420'] 


['1DMRRD_EN23X_08092023_B22'] 


['1DMRRD_EN23X_08162023_4F8'] 


Source //naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing\1DHINT_RG23XUT_LOCAL10112022_42008092023.csv 

Destination Y:\CCM_AUTOMATION\Adhoc\Output\Testing_Files_JH\1DHINT_RG23XUT_LOCAL10112022_42008092023.csv 

copied to destination successfully 


Source //naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing\1DMRRD_EN23X_08092023_B2208092023.txt 

Destination Y:\CCM_AUTOMATION\Adhoc\Output\Testing_Files_JH\1DMRRD_EN23X_08092023_B2208092023.txt 

copied to destination successfully 


Source //naomp1a_cifs.corp.intranet/dbm

In [8]:
def checkTableExists_and_CreateFile_3(dbcon):
    
    # connect to the server USODCVSQL0256,7114 at db MI_LEADS and see if there're tables in the file creation driver table
    dbcur = dbcon.cursor()
    print("Connected to SQL Server Successfully!",'\n\n')
        
    dbcur.execute("""
                  select count(*)
                  from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                  """) 
    
    if dbcur.fetchone()[0] > 0:
        print('There are files/tables in the driver table!','\n\n') 
        
        # Read table and check if column contains processed = 1 and DATETIME_PROCESSED, if contains 1 then ignore and only process processed = 0 
        # populate file creation driver table
        sql = str("""select * 
                     from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                      --where PROCESSED = 0 and DATETIME_PROCESSED is null
                      ; -- static table
                  """)
                
        df = pd.DataFrame(pd.read_sql(sql,con = dbcon))
        print(df['TACTIC'].tolist(),'\n\n')
        
        
        
        for ind, row in df.iterrows():  #print(row['PROCESSED'], row['TACTIC'], row['TABLE_NAME'])
            if (row['PROCESSED'] == 0):
                if (row['TACTIC'].strip() == 'DM'):
                    
                    # iterate rows in dataframe to identify different tactics and export files in different format
                    dm_sql = str("""select distinct TABLE_NAME
                                    from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where PROCESSED = 0 
                                    and DATETIME_PROCESSED is null
                                    and TACTIC = 'DM'
                                    and TABLE_NAME = '{0}'
                                    ; -- static table
                                 """).format(row['TABLE_NAME'])
                    dm_table_name = pd.DataFrame(pd.read_sql(dm_sql, con = dbcon))
                    print(dm_table_name['TABLE_NAME'].tolist(),'\n\n')
                    
                    # read in table and output files to source folders
                    for dm_tablename in dm_table_name['TABLE_NAME'].tolist():
                        read_dm_table_sql = str("""select * from [{0}];
                                                """).format(dm_tablename)
                        df_each_dm_table = pd.read_sql(read_dm_table_sql, con = dbcon) 
                        df_each_dm_table.to_csv('//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing/' + dm_tablename + date.today().strftime("%m%d%Y")+'.txt',  sep='|', header=True, index=False)
                    
                        # update table records with processed = 1 and system datetime stamp 
                        update_dm_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (dm_tablename))
                        dbcon.commit()
                        
                if (row['TACTIC'].strip() == 'TM'):
                    tm_sql = str("""select distinct TABLE_NAME
                                    from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where PROCESSED = 0 
                                    and DATETIME_PROCESSED is null
                                    and TACTIC = 'TM'
                                    and TABLE_NAME = '{0}'
                                    ; -- static table
                                 """).format(row['TABLE_NAME'])       
                    tm_table_name = pd.DataFrame(pd.read_sql(tm_sql, con = dbcon))
                    print(tm_table_name['TABLE_NAME'].tolist(),'\n\n')
                
                    for tm_tablename in tm_table_name['TABLE_NAME'].tolist():
                        read_tm_table_sql = str("""select * from [{0}];
                                                """).format(tm_tablename)
                        df_each_tm_table = pd.read_sql(read_tm_table_sql, con = dbcon) 
                        df_each_tm_table.to_csv('//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing/' + tm_tablename + date.today().strftime("%m%d%Y")+'.txt',  sep='?', header=True, index=False)
                    
                        # update table records with processed = 1 and system datetime stamp 
                        update_tm_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (tm_tablename))
                        dbcon.commit()   

                if (row['TACTIC'].strip() == 'EM'):
                    em_sql = str("""select distinct TABLE_NAME
                                    from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where PROCESSED = 0 
                                    and DATETIME_PROCESSED is null
                                    and TACTIC = 'EM'
                                    and TABLE_NAME = '{0}'                                    
                                    ; -- static table
                                  """).format(row['TABLE_NAME'])
                    em_table_name = pd.DataFrame(pd.read_sql(em_sql, con = dbcon))
                    print(em_table_name['TABLE_NAME'].tolist(),'\n\n')
                
                    for em_tablename in em_table_name['TABLE_NAME'].tolist():
                        read_em_table_sql = str("""select * from [{0}];
                                                """).format(em_tablename)
                        df_each_em_table = pd.read_sql(read_em_table_sql, con = dbcon) 
                        df_each_em_table.to_csv('//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing/' + em_tablename + date.today().strftime("%m%d%Y")+'.csv',  sep=',', header=True, index=False)
               
                        # update table records with processed = 1 and system datetime stamp 
                        update_em_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (em_tablename))
                        dbcon.commit()           

                        
                if (row['TACTIC'].strip() == 'D2D'):
                    D2D_sql = str("""select distinct TABLE_NAME
                                      from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                      where PROCESSED = 0 
                                      and DATETIME_PROCESSED is null
                                      and TACTIC = 'D2D'
                                      and TABLE_NAME = '{0}'   
                                      ; -- static table
                                  """).format(row['TABLE_NAME'])
                    D2D_table_name = pd.DataFrame(pd.read_sql(D2D_sql, con = dbcon))
                    print(D2D_table_name['TABLE_NAME'].tolist(),'\n\n')
                
                    for D2D_tablename in D2D_table_name['TABLE_NAME'].tolist():
                        read_d2d_table_sql = str("""select * from [{0}];
                                                 """).format(D2D_tablename)
                        df_each_d2d_table = pd.read_sql(read_d2d_table_sql, con = dbcon) 
                        df_each_d2d_table.to_csv('//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing/' + D2D_tablename + date.today().strftime("%m%d%Y")+'.csv',  sep=',', header=True, index=False)
                
                        # update table records with processed = 1 and system datetime stamp 
                        update_d2d_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (D2D_tablename))
                        dbcon.commit()   
       
        
                if (row['TACTIC'].strip() == 'DH'):
                    DH_sql = str("""select distinct TABLE_NAME
                                     from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                     where PROCESSED = 0 
                                     and DATETIME_PROCESSED is null
                                     and TACTIC = 'DH'
                                     and TABLE_NAME = '{0}'  
                                     ; -- static table
                                   """).format(row['TABLE_NAME'])
                    DH_table_name = pd.DataFrame(pd.read_sql(DH_sql, con = dbcon))
                    print(DH_table_name['TABLE_NAME'].tolist(),'\n\n')
                
                    for DH_tablename in DH_table_name['TABLE_NAME'].tolist():
                        read_dh_table_sql = str("""select * from [{0}];
                                                """).format(DH_tablename)
                            
                        df_each_dh_table = pd.read_sql(read_dh_table_sql, con = dbcon) 
                        df_each_dh_table.to_csv('//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing/' + DH_tablename + date.today().strftime("%m%d%Y")+'.csv',  sep=',', header=True, index=False)

                        # update table records with processed = 1 and system datetime stamp 
                        update_dh_table_sql = dbcur.execute ("UPDATE LEADS_LIST_FILE_CREATE_REQUEST SET PROCESSED = 1, DATETIME_PROCESSED = sysdatetime() where TABLE_NAME = ?", (DH_tablename))
                        dbcon.commit()           
       
    
            
            elif row['PROCESSED'] == 1:
                print('Table have already been processed, please check back later!')
                
            else:
                print('Table have already been processed, please check back later!')


    
    else:
        print('There are no files/tables in the driver table!')
       # return False
    
    
    # move files to destination folder #\CCM_AUTOMATION\Adhoc\Output
    src_files = os.listdir(r"//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing")
    src = r"//naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing"
    dest = r"Y:\CCM_AUTOMATION\Adhoc\Output\Testing_Files_JH" #r"C:\\Users\\AC27802\OneDrive - CenturyLink\Desktop\test_sh\copy_test_files" -- need to use mapped network drive 
     
    try:
        for file_name in src_files:
            dest_full_file_name = os.path.join(dest, file_name)
            src_full_file_name = os.path.join(src, file_name)
            if os.path.isfile(src_full_file_name):
                shutil.copy(src_full_file_name, dest)
                print("Source", src_full_file_name,'\n\n'
                      "Destination",dest_full_file_name,'\n\n'
                      "copied to destination successfully",'\n\n') 

    except IOError:
        print("Error raised with moving files to destination!",'\n\n') 
    else:
        print("Source file copied to destination successfully!",'\n\n')

   # dbcon.close() # close server connection 

In [9]:
checkTableExists_and_CreateFile_3(conn)

Connected to SQL Server Successfully! 


There are files/tables in the driver table! 


['DM ', 'DM ', 'D2D', 'EM ', 'EM ', 'TM ', 'DH ', 'DM ', 'DM '] 


Table have already been processed, please check back later!
Table have already been processed, please check back later!
Table have already been processed, please check back later!
Table have already been processed, please check back later!
Table have already been processed, please check back later!
Table have already been processed, please check back later!
Table have already been processed, please check back later!
Table have already been processed, please check back later!
Table have already been processed, please check back later!
Source //naomp1a_cifs.corp.intranet/dbmteam/Jenny_SH/txt_files_testing\1DHINT_RG23XUT_LOCAL10112022_42001122023.csv 

Destination Y:\CCM_AUTOMATION\Adhoc\Output\Testing_Files_JH\1DHINT_RG23XUT_LOCAL10112022_42001122023.csv 

copied to destination successfully 


Source //naomp1a_cifs.corp.intranet/dbmtea

# Email Notification 

##### Step1: Read table from table MI_LEADS_OUTPUT.DBO.LEADS_LIST_FILE_CREATE_REQUEST 
##### Step2: Count total by SPR_NBR for DM files 
##### Step3: Generate email based on tactics, campaign date and campaign name in the email 

In [181]:
df.head(10)
date.today().strftime("%Y-%m-%d")

,ROW_ID,TABLE_NAME,PROCESSED,DATETIME_PROCESSED,TACTIC,CAMPAIGN_NAME,VENDOR,CAMPAIGN_DATE,DISTRIBUTION_LIST
0,1,AB97883_DM_FINAL_TEST,1,2023-01-12 14:38:18.780,DM,Test Campaign,INT,2023-08-08,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
1,2,1DMRRD_EN23X_10122022_B44,1,2023-01-12 14:38:18.797,DM,eBill Notification,RRD,2023-08-08,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
2,3,1DRCYD_QW_WAD9_10062022_6C6,1,2023-01-12 14:38:24.673,D2D,None,None,None,None
3,4,1EMSMC_EN00Y_10132022,1,2023-01-12 14:38:24.703,EM,eBill Notification,SMC,2023-08-08,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
4,5,1EMSMC_UJ00Y_10132022,1,2023-01-12 14:38:24.860,EM,New_or_Upg_UO,SMC,2023-08-08,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
5,6,1TMVXI_NP24Y_10102022_145,1,2023-01-12 14:38:24.860,TM,NPS_Detractor,VXI,2023-08-08,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
6,7,1DHINT_RG23XUT_LOCAL10112022_420,1,2023-01-12 14:38:24.983,DH,UT_LOCAL_Dec,INT,2023-08-08,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
7,8,1DMRRD_EN23X_08092023_B22,1,2023-08-08 10:19:30.227,DM,eBill Notification,RRD,2023-08-09,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM
8,9,1DMRRD_EN23X_08162023_4F8,1,2023-08-08 10:19:33.053,DM,eBill Notification,RRD,2023-08-16,CRMDBA@LUMEN.COM;CLK-LEADS-TEAM@CENTURYLINK.COM


'2023-08-09'

In [178]:
def leads_email_notifications(dbcon):
       
       outlook = client.Dispatch("Outlook.Application") # to dispatch any of the office products you installed on the computer 

       for index, row in df.iterrows():
             if  (row['TACTIC'].strip()== 'DM') and (row['PROCESSED'] == 1) and (row['DATETIME_PROCESSED'] is not None):
                    # iterate rows in dataframe to identify different tactics and export files in different format
                    dm_sql = str("""select distinct TABLE_NAME
                                    from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where TACTIC = 'DM'
                                    and TABLE_NAME = '{0}'
                                    ; -- static table
                                 """).format(row['TABLE_NAME'])
                    dm_table_name = pd.DataFrame(pd.read_sql(dm_sql, con = dbcon))
                    #print(dm_table_name['TABLE_NAME'].tolist(),'\n\n')
                    
                    # read in table and output files to source folders
                    for dm_tablename in dm_table_name['TABLE_NAME'].tolist():
                        read_dm_table_sql = str("""select SPR_NBR as KEYCODE, count(*) as COUNT
                                                   from MI_LEADS_OUTPUT.[dbo].[{0}]
                                                   group by SPR_NBR;
                                                """).format(dm_tablename)
                        df_each_dm_table = pd.read_sql(read_dm_table_sql, con = dbcon) 
                        dm_html_table = df_each_dm_table.to_html(index=False) # transform table in html format

                        # send email notifications
                        mail = outlook.CreateItem(0)
                        mail.To = row["DISTRIBUTION_LIST"]
                        mail.Subject = f"{row['CAMPAIGN_NAME']} {row['TACTIC']} LIST {row['CAMPAIGN_DATE']} ct:{df_each_dm_table.COUNT.sum()}"
                        mail.HTMLBody = f"""
                                            <b>Team, <br><br>
                                            I am sending the {row['CAMPAIGN_NAME']} leads to you via ftp. The details of the file are listed below. <br><br>
                                            Counts for this campaign are provided below.<br><br>
                                            File name:<br><br>
                                            ---------------------------------------------------<br><br>
                                            {dm_tablename}.TXT  - {df_each_dm_table.COUNT.sum()} records <br><br>
                                            {dm_html_table}
                                            
                                         """

                        #mail.Display()
                        mail.Send()


             if  (row['TACTIC'].strip()== 'TM') and (row['PROCESSED'] == 1) and (row['DATETIME_PROCESSED'] is not None):
                    # iterate rows in dataframe to identify different tactics and export files in different format
                    tm_sql = str("""select distinct TABLE_NAME
                                    from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where TACTIC = 'TM'
                                    and TABLE_NAME = '{0}'
                                    ; -- static table
                                 """).format(row['TABLE_NAME'])
                    tm_table_name = pd.DataFrame(pd.read_sql(tm_sql, con = dbcon))
                    #print(tm_table_name['TABLE_NAME'].tolist(),'\n\n')
                    
                    # read in table and output files to source folders
                    for tm_tablename in tm_table_name['TABLE_NAME'].tolist():
                        read_tm_table_sql = str("""select CAMPAIGN_CODE as KEYCODE, count(*) as COUNT
                                                   from MI_LEADS_OUTPUT.[dbo].[{0}]
                                                   group by CAMPAIGN_CODE;
                                                """).format(tm_tablename)
                        df_each_tm_table = pd.read_sql(read_tm_table_sql, con = dbcon) 
                        tm_html_table = df_each_tm_table.to_html(index=False) # transform table in html format

                        # send email notifications
                        mail = outlook.CreateItem(0)
                        mail.To = row["DISTRIBUTION_LIST"]
                        mail.Subject = f"{row['CAMPAIGN_NAME']} {row['TACTIC']} LIST {row['CAMPAIGN_DATE']} ct:{df_each_tm_table.COUNT.sum()}"
                        mail.HTMLBody = f"""
                                            <b>Team, <br><br>
                                            I am sending the {row['CAMPAIGN_NAME']} leads to you via ftp. The details of the file are listed below. <br><br>
                                            Counts for this campaign are provided below.<br><br>
                                            File name:<br><br>
                                            ---------------------------------------------------<br><br>
                                            {tm_tablename}.csv - {df_each_tm_table.COUNT.sum()} records <br><br>
                                            {tm_html_table}
                                            
                                         """
                        #mail.Display()
                        mail.Send()


             if  (row['TACTIC'].strip()== 'EM') and (row['PROCESSED'] == 1) and (row['DATETIME_PROCESSED'] is not None):
                    # iterate rows in dataframe to identify different tactics and export files in different format
                    em_sql = str("""select distinct TABLE_NAME
                                    from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where TACTIC = 'EM'
                                    and TABLE_NAME = '{0}'
                                    ; -- static table
                                 """).format(row['TABLE_NAME'])
                    em_table_name = pd.DataFrame(pd.read_sql(em_sql, con = dbcon))
                    #print(em_table_name['TABLE_NAME'].tolist(),'\n\n')
                    
                    # read in table and output files to source folders
                    for em_tablename in em_table_name['TABLE_NAME'].tolist():
                        read_em_table_sql = str("""select CAMPAIGN_CODE as KEYCODE, count(*) as COUNT
                                                   from MI_LEADS_OUTPUT.[dbo].[{0}]
                                                   group by CAMPAIGN_CODE;
                                                """).format(em_tablename)
                        df_each_em_table = pd.read_sql(read_em_table_sql, con = dbcon) 
                        em_html_table = df_each_em_table.to_html(index=False) # transform table in html format

                        # send email notifications
                        mail = outlook.CreateItem(0)
                        mail.To = row["DISTRIBUTION_LIST"]
                        mail.Subject = f"{row['CAMPAIGN_NAME']} {row['TACTIC']} LIST {row['CAMPAIGN_DATE']} ct:{df_each_em_table.COUNT.sum()}"
                        mail.HTMLBody = f"""
                                            <b>Team, <br><br>
                                            I am sending the {row['CAMPAIGN_NAME']} leads to you via ftp. The details of the file are listed below. <br><br>
                                            Counts for this campaign are provided below.<br><br>
                                            File name:<br><br>
                                            ---------------------------------------------------<br><br>
                                            {em_tablename}.csv - {df_each_em_table.COUNT.sum()} records <br><br>
                                            {em_html_table}
                                            
                                         """
                        #mail.Display()
                        mail.Send()


             if  (row['TACTIC'].strip()== 'DH') and (row['PROCESSED'] == 1) and (row['DATETIME_PROCESSED'] is not None):
                    # iterate rows in dataframe to identify different tactics and export files in different format
                    dh_sql = str("""select distinct TABLE_NAME
                                    from MI_LEADS_OUTPUT.[dbo].[LEADS_LIST_FILE_CREATE_REQUEST]
                                    where TACTIC = 'DH'
                                    and TABLE_NAME = '{0}'
                                    ; -- static table
                                 """).format(row['TABLE_NAME'])
                    dh_table_name = pd.DataFrame(pd.read_sql(dh_sql, con = dbcon))
                    #print(dh_table_name['TABLE_NAME'].tolist(),'\n\n')
                    
                    # read in table and output files to source folders
                    for dh_tablename in dh_table_name['TABLE_NAME'].tolist():
                        read_dh_table_sql = str("""select SPR_NBR as KEYCODE, count(*) as COUNT
                                                   from MI_LEADS_OUTPUT.[dbo].[{0}]
                                                   group by SPR_NBR;
                                                """).format(dh_tablename)
                        df_each_dh_table = pd.read_sql(read_dh_table_sql, con = dbcon) 
                        dh_html_table = df_each_dh_table.to_html(index=False) # transform table in html format

                        # send email notifications
                        mail = outlook.CreateItem(0)
                        mail.To = row["DISTRIBUTION_LIST"]
                        mail.Subject = f"{row['CAMPAIGN_NAME']} {row['TACTIC']} LIST {row['CAMPAIGN_DATE']} ct:{df_each_dh_table.COUNT.sum()}"
                        mail.HTMLBody = f"""
                                            <b>Team, <br><br>
                                            I am sending the {row['CAMPAIGN_NAME']} leads to you via ftp. The details of the file are listed below. <br><br>
                                            Counts for this campaign are provided below.<br><br>
                                            File name:<br><br>
                                            ---------------------------------------------------<br><br>
                                            {dh_tablename}.csv - {df_each_dh_table.COUNT.sum()} records <br><br>
                                            {dh_html_table}
                                            
                                         """
                        #mail.Display()
                        mail.Send()

               

In [179]:
leads_email_notifications(conn)